In [ ]:
kbase: cd 'C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\Internships\Agronne\cobrakbase'
modelseedpy: cd 'C:\Users\Andrew Freiburger\Dropbox\My PC (DESKTOP-M302P50)\Documents\UVic Civil Engineering\Internships\Agronne\ModelSEEDpy'
    
CPLEX: python 'C:\Program Files\IBM\ILOG\CPLEX_Studio_Community201\python\setup.py' install
    
    'C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.8_3.8.2800.0_x64__qbz5n2kfra8p0\Lib\site-packages\cplex':
    
# KBase token = SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF

# Import statements

In [34]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
import json
import sys
import cProfile
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobra.flux_analysis.parsimonious import pfba
from IPython.core.display import HTML
import os
import re

os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
#sys.path.append("/Users/chenry/code/ModelSEEDpy")
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase = cobrakbase.KBaseAPI(token)

# import modelseedpy.fbapkg
import modelseedpy
from modelseedpy.biochem import from_local
from modelseedpy import ElementUptakePkg, SimpleThermoPkg, ReactionUsePkg, BilevelPkg, CommKineticPkg,KBaseMediaPkg


modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
bigg_model_path = '..\COBRA function scripts\e_coli_core metabolism from BiGG.json'
model = cobra.io.load_json_model(bigg_model_path)
modelseed = modelseedpy.biochem.from_local(modelseed_path)

In [72]:
# import the kbase narrative model
object_json = kbase.get_object('E_iAH991V2', 93832)
#dir(object_json)
'''for obj in object_json['modelreactions']:
    print(obj['modelReactionReagents'])'''

import re 

#dir(modelseed)
'''for reaction in modelseed.reactions:
    print(reaction)'''

reactions = {}
for reaction in object_json['modelreactions']:
    reagents = {}
    reaction_id = reaction['id']
    reaction_name = reaction['name']
    stoich = reaction['modelReactionReagents']
    for reagent in stoich:
        # parse the compound ids and the compound compartments
        modelseed_id = re.search('(?<=\/)([^\/]+$)', reagent['modelcompound_ref']).group(1)
        compartment = re.search('(_[a-z][0-9])', modelseed_id).group(1)
        compartment = re.sub('(_)', '', compartment)
        modelseed_id = re.sub('(_[a-z][0-9])', '', modelseed_id)
        
        # create the reagent dictionary
        reagents[modelseed_id] = {'coefficient': reagent['coefficient'], 'compartment': compartment}
        
    # create the reaction dictionary
    reactions[reaction_id] = {'name': reaction_name,
                              'reagents': reagents}

for reaction, information in reactions.items():
    for reagent in information['reagents']:
        if re.search('cpd', reagent):
            print(modelseed.get_seed_compound(reagent).data)
    
#print(object_json.keys()) # keys
#print(len(object_json['modelcompounds'])) # total compounds 1179

#unique_mcs = dict((x['id'], x) for x in object_json['modelcompounds']) 

#print(len(unique_mcs)) # total compounds 1178 we lost a compound because it was a duplicate, which caused a hashkey collision

#object_json['modelcompounds'] = list(unique_mcs.values()) # update data without the duplicate id
#kbase.save_object('E_iAH991V2', 'freiburgermsu:narrative_1624557251879', 'KBaseFBA.FBAModel', object_json) # saving the object back to KBase with id=E_iAH991V2 to the workspace freiburgermsu:narrative_1624557251879 assigning this type KBaseFBA.FBAModel and with data=object_json

{'id': 'cpd01861', 'abbreviation': '12ppd-R', 'name': '(R)-1,2-Propanediol', 'formula': 'C3H8O2', 'mass': '76.0', 'source': 'Primary Database', 'inchikey': 'DNIAPMSPPWPWGF-GSVOUGTGSA-N', 'charge': 0, 'is_core': 1, 'is_obsolete': 0, 'linked_compound': nan, 'is_cofactor': 0, 'deltag': -1.42, 'deltagerr': 0.24, 'pka': '1:4:16.00;1:5:14.47', 'pkb': '1:4:-2.86;1:5:-2.93', 'abstract_compound': nan, 'comprised_of': nan, 'aliases': 'Name: (R)-1,2-Propanediol; (R)-Propane-1,2-diol; (R)-Propylene glycol; (R)-propane-1,2-diol; D-1,2-propanediol|BiGG: 12ppd__R|EcoCyc: CPD-8891|KEGG: C02912|MetaCyc: CPD-8891|iAF1260: 12ppd-R', 'smiles': 'C[C@@H](O)CO'}
{'id': 'cpd00453', 'abbreviation': '12ppd-S', 'name': '1,2-Propanediol', 'formula': 'C3H8O2', 'mass': '76.0', 'source': 'Primary Database', 'inchikey': 'DNIAPMSPPWPWGF-UHFFFAOYSA-N', 'charge': 0, 'is_core': 1, 'is_obsolete': 0, 'linked_compound': nan, 'is_cofactor': 0, 'deltag': -2.76, 'deltagerr': 0.45, 'pka': '1:4:16.00;1:5:14.47', 'pkb': '1:4:-2.8

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
# import general modules
import cobra
import cplex
import sys
import os

# import cobrakbase
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase_api = cobrakbase.KBaseAPI(token)

# import modelseed functions
from modelseedpy import KBaseMediaPkg
from modelseedpy import FBAHelper, MSBuilder, MSGenome
from modelseedpy.biochem import from_local
modelseed_path = '..\..\..\Biofilm growth code\GSWL code\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)

# define the kbase API 

genome = MSGenome.from_fasta('GCF_000005845.2_ASM584v2_protein.faa', split=' ')
print('Number of features:', len(genome.features))
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
#template = kbase_api.get_from_ws("GramPosModelTemplateV3","NewKBaseModelTemplates")

#The first argument is the model ID and can be anything you want
model = MSBuilder.build_metabolic_model('ecoli', genome, template, allow_all_non_grp_reactions=True)

#Pulling a KBase media to specify an environment for gapfilling
media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")

#Gapfilling the model in the specified media
model = MSBuilder.gapfill_model(model,"bio1",template,media)

#Applying the same media to the resulting model
kmp = KBaseMediaPkg(model)
kmp.build_package(media)

#Running FBA to demonstrate growth
sol=model.optimize()
model.summary()

FileNotFoundError: [Errno 2] No such file or directory: 'GCF_000005845.2_ASM584v2_protein.faa'

In [14]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
import json
import sys
import cProfile
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobra.flux_analysis.parsimonious import pfba
from IPython.core.display import HTML
import os
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
#sys.path.append("/Users/chenry/code/ModelSEEDpy")
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase_api = cobrakbase.KBaseAPI(token)

# import modelseedpy.fbapkg
from modelseedpy import ElementUptakePkg, SimpleThermoPkg, ReactionUsePkg, BilevelPkg, CommKineticPkg,KBaseMediaPkg

# model definitions 
model_information = {}

''' Community models
possible_models = ['CMM_iAH991V2_iML1515.kb', 'electrosynth_comnty.mdl.gf.2021']
possible_models_ids = [40576, 93204]
possible_models_media = ['Btheta_Ecoli_minimal_media', 'CO2_minimal']
'''

''' Individual models'''
possible_models_names = ['E_iAH991V2', 'E_iML1515.kb']
possible_models_ids = [40576, 40576]
possible_models_media = ['Btheta_Ecoli_minimal_media', 'Btheta_Ecoli_minimal_media']
for possible_model in possible_models:
    index = possible_models.index(possible_model)
    model_information[possible_model] = {'model': possible_models_names[index],
                                         'id_number': possible_models_ids[index],
                                         'media': possible_models_media[index]}

dataset_index = 0 
model_name = list(model_information.keys())[dataset_index]
model_id = list(model_information.values())[dataset_index]['id_number']
model = kbase_api.get_from_ws(model_name, model_id)
model.solver = 'optlang-cplex'

#Applying uptake limit to total carbon uptake by model
eup = ElementUptakePkg(model)
eup.build_package({"C":60})

#Applying constraints based on KBase media
model_media = list(model_information.values())[dataset_index]['media']
media = kbase_api.get_from_ws(model_media, model_id)
kmp = KBaseMediaPkg(model)
kmp.build_package(media)

#Applying community model kinetic constraints
ckp = CommKineticPkg(model)
ckp.build_package(2100,{"1":0.5,"2":0.5})

biomass_objective = model.problem.Objective(
    1 * model.reactions.bio1.flux_expression,
    direction='max')
model.objective = biomass_objective

sol = model.optimize()
model.summary()

ValueError: id 'ggdp_c0' at index 893 is non-unique. Is it present twice?

# Brainstorming

### Annotated suggestion from Filipe

In [18]:
import cobra
from cobra import Model, Reaction, Metabolite
import pandas as pd
import cplex
import optlang
import math
import json
import sys
import cProfile
from optlang.symbolics import Zero, add
import cobra.util.solver as sutil
from cobra.flux_analysis.parsimonious import pfba
from IPython.core.display import HTML
import os
os.environ["HOME"] = 'C:\\Users\\Andrew Freiburger\\Dropbox\\My PC (DESKTOP-M302P50)\\Documents\\UVic Civil Engineering\\Internships\\Agronne\\cobrakbase'
import cobrakbase
#sys.path.append("/Users/chenry/code/ModelSEEDpy")
token = 'SF3CW6PH4CKWSRMOA3JQPSGC6YERTRSF'
kbase = cobrakbase.KBaseAPI(token)

# import modelseedpy.fbapkg
from modelseedpy import ElementUptakePkg, SimpleThermoPkg, ReactionUsePkg, BilevelPkg, CommKineticPkg,KBaseMediaPkg

object_json = kbase.get_object('E_iAH991V2', 93832)

print(type(object_json)) # objects json are dict type
print(object_json.keys()) # keys
print(len(object_json['modelcompounds'])) # total compounds 1179

unique_mcs = dict((x['id'], x) for x in object_json['modelcompounds']) 

print(len(unique_mcs)) # total compounds 1178 we lost a compound because it was a duplicate, which caused a hashkey collision

object_json['modelcompounds'] = list(unique_mcs.values()) # update data without the duplicate id
#kbase.save_object('E_iAH991V2', 'freiburgermsu:narrative_1624557251879', 'KBaseFBA.FBAModel', object_json) # saving the object back to KBase with id=E_iAH991V2 to the workspace freiburgermsu:narrative_1624557251879 assigning this type KBaseFBA.FBAModel and with data=object_json

<class 'dict'>
dict_keys(['__VERSION__', 'biomasses', 'delete_biomasses', 'deleted_reactions', 'gapfilledcandidates', 'gapfillings', 'gapgens', 'genome_ref', 'id', 'model_edits', 'modelcompartments', 'modelcompounds', 'modelreactions', 'name', 'quantopts', 'source', 'source_id', 'template_ref', 'template_refs', 'type'])
1179
1178


In [1]:
import cplex
import os

os.path.dirname(cplex.__file__)

'C:\\Program Files (x86)\\Anaconda3\\lib\\site-packages\\cplex'

In [70]:
dir(modelseed)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'compound_aliases',
 'compound_structures',
 'compounds',
 'get_attribute',
 'get_formula',
 'get_mapping_external_to_seed',
 'get_name',
 'get_non_obsolete',
 'get_seed_compound',
 'get_seed_compound_by_alias',
 'get_seed_reaction',
 'get_seed_reaction_by_alias',
 'reaction_aliases',
 'reaction_ecs',
 'reactions',
 'summary']

In [51]:
import modelseedpy

#dir(modelseedpy)
#dir(object_json)

for compound in object_json['modelcompounds']:
    print(compound)

{'aliases': ['mdlid:12ethd'], 'charge': 0, 'compound_ref': '~/template/compounds/id/cpd00000', 'dblinks': {}, 'formula': '', 'id': '12ethd_e0', 'modelcompartment_ref': '~/modelcompartments/id/e0', 'name': '12ethd_e0', 'numerical_attributes': {}, 'string_attributes': {}}
{'aliases': ['mdlid:12ethd'], 'charge': 0, 'compound_ref': '~/template/compounds/id/cpd00000', 'dblinks': {}, 'formula': '', 'id': '12ethd_c0', 'modelcompartment_ref': '~/modelcompartments/id/c0', 'name': '12ethd_c0', 'numerical_attributes': {}, 'string_attributes': {}}
{'aliases': ['mdlid:12ppd_R'], 'charge': 0, 'compound_ref': '~/template/compounds/id/cpd01861', 'dblinks': {}, 'formula': 'C3H8O2', 'id': 'cpd01861_e0', 'modelcompartment_ref': '~/modelcompartments/id/e0', 'name': '(R)-1,2-Propanediol_e0', 'numerical_attributes': {}, 'string_attributes': {}}
{'aliases': ['mdlid:12ppd_R'], 'charge': 0, 'compound_ref': '~/template/compounds/id/cpd01861', 'dblinks': {}, 'formula': 'C3H8O2', 'id': 'cpd01861_c0', 'modelcompar

In [66]:
import re 

#dir(modelseed)

'''for reaction in modelseed.reactions:
    print(reaction)'''

reactions = {}
for reaction in object_json['modelreactions']:
    reagents = {}
    reaction_id = reaction['id']
    reaction_name = reaction['name']
    stoich = reaction['modelReactionReagents']
    for reagent in stoich:
        # parse the compound ids and the compound compartments
        modelseed_id = re.search('(?<=\/)([^\/]+$)', reagent['modelcompound_ref']).group(1)
        compartment = re.search('(_[a-z][0-9])', modelseed_id).group(1)
        compartment = re.sub('(_)', '', compartment)
        modelseed_id = re.sub('(_[a-z][0-9])', '', modelseed_id)
        
        # create the reagent dictionary
        reagents[modelseed_id] = {'coefficient': reagent['coefficient'], 'compartment': compartment}
        
    # create the reaction dictionary
    reactions[reaction_id] = {'name': reaction_name,
                              'reagents': reagents}

for reaction in reactions:
    print(reaction, ': ', reactions[reaction])

#dir(reaction)

12ETHDt_c0 :  {'name': '1,2-Ethanediol facilitated transport', 'reagents': {'12ethd': {'coefficient': -1, 'compartment': 'e0'}}}
12PPDRt_c0 :  {'name': 'R-Propane-1,2-diol facilitated transport', 'reagents': {'cpd01861': {'coefficient': 1, 'compartment': 'c0'}}}
12PPDt_c0 :  {'name': 'S-Propane-1,2-diol facilitated transport', 'reagents': {'cpd00453': {'coefficient': 1, 'compartment': 'c0'}}}
23PDE2_c0 :  {'name': '2,3-Cyclic UMP 3-nucleotidohydrolase', 'reagents': {'cpd01572': {'coefficient': -1, 'compartment': 'c0'}, 'cpd00001': {'coefficient': -1, 'compartment': 'c0'}, 'cpd00989': {'coefficient': 1, 'compartment': 'c0'}, 'cpd00067': {'coefficient': 1, 'compartment': 'c0'}}}
23PDE4_c0 :  {'name': '2,3-Cyclic CMP 3-nucleotidohydrolase', 'reagents': {'cpd00001': {'coefficient': -1, 'compartment': 'c0'}, 'cpd00067': {'coefficient': 1, 'compartment': 'c0'}, 'cpd01571': {'coefficient': -1, 'compartment': 'c0'}, 'cpd03454': {'coefficient': 1, 'compartment': 'c0'}}}
23PDE7_c0 :  {'name': '2